In [ ]:
import ollama
import psycopg2
from datetime import datetime

# Database connection
def connect_db():
    return psycopg2.connect(
        dbname="your_db",
        user="your_user",  # Change if using a different user
        password="your_pass",  # Set your PostgreSQL password
        host="localhost",
        port="5432"
    )

# Function to generate the first chatbot message
def start_conversation(learning_language, proficiency):
    model = "mistral"

    prompt = f"""
    You are a friendly tutor helping someone practice {learning_language}.
    Their proficiency level is {proficiency}.
    Start a conversation with them by saying something natural in {learning_language}.
    Keep it short and engaging. Also make the conversation flow in a natural way.
    Point out users mistakes and make each conversation short to medium length and don't overload them with multiple tips.
    """

    response = ollama.chat(model=model, messages=[{"role": "user", "content": prompt}])
    return response['message']['content']

# Function to handle the conversation
def chatbot_conversation(learning_language, proficiency):
    model = "mistral"
    conversation = []

    chatbot_message = start_conversation(learning_language, proficiency)
    print("\nChatbot:", chatbot_message)

    for turn in range(5):  # 5 chatbot turns, 5 user turns
        user_input = input("\nYou: ")
        conversation.append({"role": "user", "content": user_input})

        response = ollama.chat(model=model, messages=conversation)
        chatbot_message = response['message']['content']
        print("\nChatbot:", chatbot_message)

        conversation.append({"role": "assistant", "content": chatbot_message})

    return conversation

# Function to analyze mistakes
def analyze_mistakes(conversation, learning_language):
    model = "mistral"
    
    user_responses = [msg['content'] for msg in conversation if msg['role'] == "user"]
    full_conversation = "\n".join([f"User: {msg['content']}" for msg in conversation if msg['role'] == "user"])

    prompt = f"""
    You are an expert {learning_language} tutor.
    Analyze the user's responses in the conversation below.
    Identify and explain any grammar, vocabulary, or pronunciation mistakes.
    Provide a corrected version for each mistake and after that also explain to the user on which points they should improve upon.
    Here is the user's conversation:\n{full_conversation}
    """

    response = ollama.chat(model=model, messages=[{"role": "user", "content": prompt}])
    return response['message']['content']

# Function to save user progress to PostgreSQL
def save_to_db(user_id, known_languages, learning_language, proficiency, conversation, mistakes):
    conn = connect_db()
    cursor = conn.cursor()

    cursor.execute("""
        INSERT INTO user_progress (user_id, known_languages, learning_language, proficiency, conversation, mistakes, timestamp)
        VALUES (%s, %s, %s, %s, %s, %s, %s)
    """, (user_id, known_languages, learning_language, proficiency, conversation, mistakes, datetime.now()))

    conn.commit()
    cursor.close()
    conn.close()

# Main chatbot flow
def main():
    print("Welcome to the Language Learning Chatbot!\n")

    # Get user input
    user_id = input("Enter your user ID: ").strip()
    known_languages = input("Enter the languages you know (comma-separated): ").strip()
    learning_language = input("Enter the language you want to learn: ").strip()
    proficiency = input("Enter your proficiency level (Beginner, Intermediate, Advanced): ").strip()

    # Conduct conversation
    conversation = chatbot_conversation(learning_language, proficiency)

    # Analyze mistakes
    feedback = analyze_mistakes(conversation, learning_language)
    print("\n--- Language Feedback ---")
    print(feedback)

    # Save to database
    save_to_db(user_id, known_languages, learning_language, proficiency, str(conversation), feedback)

    print("\nYour progress has been saved! ✅")

# Run the chatbot
if __name__ == "__main__":
    main()

Welcome to the Language Learning Chatbot!


Chatbot:  Hello there! I hope you're having a fantastic day. I've just been reading this interesting article about quantum computing, and it occurred to me how much our world is evolving at such a rapid pace. By the way, I noticed that you used "rapid" instead of "rapidly". It's great that you're taking English practice seriously!

   So, speaking of advancements, what do you think about the concept of artificial intelligence surpassing human intelligence? Do you believe it will happen in our lifetime, and how might that change the world as we know it? Let's discuss!

Chatbot:  Absolutely! As we continue to develop and integrate Artificial Intelligence (AI) into various aspects of our society, it's crucial that we focus on creating systems that align with human values and ethical standards. This approach, often referred to as Human-Centered AI or Ethical AI, ensures that technology serves the needs and respects the rights of individuals.

Her